### Step 11.  Summaries of Summaries - for OpenAI

### Import initial libraries and import the submissions CSV that includes sentiment scores

In [2]:
import pandas as pd
from threading import Lock
import concurrent.futures




In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

comments_tokenizer = AutoTokenizer.from_pretrained("stevied67/pegasus-subreddit-comments-summarizer")


#### Functions

In [3]:


def concat_comments(df):
    # Group the DataFrame by the 'topic' column
    grouped = df.groupby('topic')

    # Initialize the new DataFrame with empty lists
    new_df = pd.DataFrame({'topic': [], 'summary': []})

    # Iterate through each group in the grouped DataFrame
    for group_name, group_df in grouped:
        # Concatenate the 'summary' values in the group with '~~' separator
        summary_list = group_df['summary'].tolist()
        summary_concatenated = []
        for summary in summary_list:
            # Check if the concatenated summary would exceed 800 tokens
            tokens_summary = comments_tokenizer.encode('~~'.join(summary_concatenated), add_special_tokens=False)
            tokens_new_summary = comments_tokenizer.encode(summary, add_special_tokens=False)
            if len(tokens_summary) + len(tokens_new_summary) > 800:
                # Add the current concatenated summary to the new DataFrame
                summary_str = '~~'.join(summary_concatenated).strip()
                new_df = pd.concat([new_df, pd.DataFrame({'topic': [group_name], 'summary': [summary_str]})])
                summary_concatenated = []
            # Concatenate the current summary to the concatenated summary with '~~' separator
            summary_concatenated.append(summary)
        
        # Add the last concatenated summary to the new DataFrame
        if summary_concatenated:
            summary_str = '~~'.join(summary_concatenated).strip()
            new_df = pd.concat([new_df, pd.DataFrame({'topic': [group_name], 'summary': [summary_str]})])

    return new_df

In [10]:
import openai
import json 
openai.api_key = "XXXXXXXXXX"

In [11]:
import json
import openai

error_string = "{\"summary\": \"none\"}"

def getOpenAI_Summary(prompt, selftext):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": "" + prompt + ": " + selftext+""}
        ]
    )

    initial_response = response['choices'][0]['message']['content']

    #print(initial_response)

    return initial_response


In [5]:
comments_df = pd.read_csv('tfcc_comments_with_accuracy.csv')


In [6]:
comments_df

,topic,selftext,summary,bert_f1
0,0.0,"Where I work, we're supposed to somewhat lay d...",Advice given:\n- Use a polite approach with in...,0.819973
1,0.0,"Not sure why I was reminded of this, but my gr...","1. When dealing with obnoxious customers, don'...",0.813728
2,0.0,"""Oh, mensa. Isn't that the private organizatio...",Advice given in the comments includes: \n1. Do...,0.813154
3,0.0,Oh thats awful. Im assuming there were rules a...,1. Be empathetic to lonely customers.\n2. Don'...,0.812379
4,0.0,I never understand ranting at call centre peop...,Advice: Be respectful and kind to call centre ...,0.837029
...,...,...,...,...
105,19.0,"I don't remember the context, but somebody was...",Advice: \n- Use empathy statements to de-escal...,0.819383
106,19.0,"So true, I had to call a customer back yesterd...",Advice:\n- Always follow protocol and stick to...,0.812546
107,19.0,"From the other side of the fence, I just had a...",Advice:\n\n- Deescalate situations with angry ...,0.809660
108,19.0,"ME: ""Thank you for calling &lt;large insurance...",Advice given:\n-Don't take it personally when ...,0.821800


#### Consolidate the comments by topic

In [7]:
filtered_df = concat_comments(comments_df)

In [8]:
filtered_df

,topic,summary
0,0.0,Advice given:\n- Use a polite approach with in...
0,1.0,Advice:\n- Watch your money closely and don't ...
0,1.0,"Advice given:\n- For utility companies, custom..."
0,2.0,Advice given:\n-call center employees should n...
0,2.0,Advice given:\n- Prices can change normally an...
0,3.0,Advice given in these comments includes:\n\n- ...
0,3.0,Advice: \n1. Don't assume that someone can ins...
0,4.0,1. The system is screwed up when it comes to b...
0,5.0,1. Caller needs to understand billing cycle.\n...
0,5.0,Advice given in these comments include:\n\n- D...


#### Generate the summaries of summaries for comments

In [ ]:
prompt = 'summarize the advice given in the following text in a minimum of 150 words:'
count = 1

new_filtered_df = filtered_df.copy()

# Create a new list to store the summaries
new_summaries = []

for index, row in new_filtered_df.iterrows():
    rtext = row["summary"]
    rtitle = ""

    rresponse = getOpenAI_Summary(prompt, rtext)

    print(count)
    count = count + 1

    # Append the summary to the list
    new_summaries.append(rresponse)

# Assign the list of summaries to the DataFrame
new_filtered_df["new_summary"] = new_summaries



In [13]:
new_filtered_df

,topic,summary,new_summary
0,0.0,Advice given:\n- Use a polite approach with in...,The article provides several pieces of advice ...
0,1.0,Advice:\n- Watch your money closely and don't ...,The text provides different pieces of advice f...
0,1.0,"Advice given:\n- For utility companies, custom...",The article provides various pieces of advice ...
0,2.0,Advice given:\n-call center employees should n...,The advice given covers a wide range of topics...
0,2.0,Advice given:\n- Prices can change normally an...,Customers should take note that prices can var...
0,3.0,Advice given in these comments includes:\n\n- ...,The advice given in the comments covers a wide...
0,3.0,Advice: \n1. Don't assume that someone can ins...,The given text provides advice on various aspe...
0,4.0,1. The system is screwed up when it comes to b...,The advice given in the text and comments cove...
0,5.0,1. Caller needs to understand billing cycle.\n...,The given text includes a collection of advice...
0,5.0,Advice given in these comments include:\n\n- D...,"The given text offers a range of advice, compl..."


In [14]:
# Function to count the number of words in a given text
def count_words(text):
    return len(text.split())

summary_word_counts = new_filtered_df['summary'].apply(count_words)
new_summary_word_counts = new_filtered_df['new_summary'].apply(count_words)

summary_avg_word_count = summary_word_counts.mean()
new_summary_avg_word_count = new_summary_word_counts.mean()

print(f"Average word count in 'summary' column: {summary_avg_word_count}")
print(f"Average word count in 'new_summary' column: {new_summary_avg_word_count}")

Average word count in 'summary' column: 411.84375
Average word count in 'new_summary' column: 178.78125


#### Export the consolidated summaries of commments with theme to a csv

In [15]:
new_filtered_df.to_csv('tfcc_comments_summaries_of_summaries.csv', index=False)

#### Create summaries of summaries for submissions.

#### Read in the submissions data that has passed accuracy checks

In [6]:
#new_filtered_df.to_csv('tfcc_submissions_top20_with_sentiment_including_comment_sentiment_and_summaries.csv', index=False)
df = pd.read_csv('tfcc_submissions_with_accuracy.csv')


In [7]:
df

,id,title,selftext,author,score,num_comments,created_date,selftext_length,topic,pos_sentiment,neg_sentiment,comments_pos_sentiment,comments_neg_sentiment,summary,bert_f1
0,acncqg,I swear I’m not usually this dumb: when the ca...,my absolute favorite type of calls are when th...,QuoteTheKitty,2002,63,2019-01-04 22:13:45,173,0,0.360828,0.639172,0.239294,0.760706,A customer telling a customer service represen...,0.836387
1,js29up,The mute button is not the customers friend,at a previous call center i worked they wanted...,supersizedlady,1756,115,2020-11-11 05:30:44,240,0,0.003723,0.996277,0.305934,0.694066,The writer worked in a call center and preferr...,0.839418
2,e01rb8,One of my agents actually said what everyone t...,this happened a couple of weeks ago and is bot...,wirwarennamenlos,1553,121,2019-11-22 14:33:25,138,0,0.498418,0.501582,0.250270,0.749730,"An agent was recorded responding with ""how the...",0.874467
3,h0gvxc,Customer loves his analogy until I use it agai...,this call was from a while ago so i don't real...,BostonB96,1411,42,2020-06-10 18:28:42,330,0,0.184346,0.815654,0.001718,0.998282,A software provider received a call from a cus...,0.845931
4,j101m6,Finally cracked on a caller.,i was barely awake enough to speak to humans b...,AgentGerk,1407,113,2020-09-27 22:15:44,527,0,0.323572,0.676428,0.135200,0.864800,The author describes an unpleasant interaction...,0.819194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1938,4oloj6,Shooter threat,"wednesday night, someone called in and threate...",Believeinthis,18,20,2016-06-17 22:08:20,149,19,0.001418,0.998582,0.001219,0.998781,An unknown person called in and threatened to ...,0.911196
1939,2dbpxp,How come people assume call center employees a...,i've had a few infuriating calls. people just ...,lacquerqueen,15,21,2014-08-12 11:05:09,254,19,0.001012,0.998988,0.150063,0.849937,The author is frustrated with the assumption t...,0.843933
1940,4x4pjt,Probably getting a CA because I didn't apologi...,customer complained that i blew her off despi...,evosthunder,14,4,2016-08-10 22:06:34,104,19,0.013253,0.986747,NaN,NaN,A customer complained about being blown off de...,0.885685
1941,2ldeab,Don't Feed Stray Cats,"about two years ago, an old lady got upset and...",EveryoneHatesCJ,7,3,2014-11-05 15:11:42,125,19,0.005375,0.994625,NaN,NaN,"An old lady, upset about an issue, requested t...",0.912156


#### Combine summaries by topic for consolidated summarization

In [ ]:
# Group the tfcc_comments_with_topics DataFrame by the 'topic' column
grouped = df.groupby('topic')

# Initialize the new DataFrame with empty lists
new_df = pd.DataFrame({'topic': [], 'summary': [], 'title': []})

# Iterate through each group in the grouped DataFrame
for group_name, group_df in grouped:
    # Concatenate the 'summary' and 'emotion' values in the group with '~~' separator
    summary_list = group_df['summary'].tolist()
    emotion_list = group_df['title'].tolist()
    summary_concatenated = []
    emotion_concatenated = []
    for summary, emotion in zip(summary_list, emotion_list):
        # Check if the concatenated summary would exceed 2000 words
        words_summary = ' '.join(summary_concatenated).split()
        if len(words_summary) + len(summary.split()) > 2000:
            # Add the current concatenated summary and emotion to the new DataFrame
            summary_str = '~~'.join(summary_concatenated).strip()
            emotion_str = '~~'.join(emotion_concatenated).strip()
            new_df = new_df.append({'topic': group_name, 'summary': summary_str, 'title': emotion_str}, ignore_index=True)
            summary_concatenated = []
            emotion_concatenated = []
        # Concatenate the current summary and emotion to the concatenated summary and emotion with '~~' separator
        summary_concatenated.append(summary)
        emotion_str = '~~'.join(emotion) if isinstance(emotion, list) else emotion
        emotion_concatenated.append(emotion_str)
    # Add any remaining concatenated summary and emotion to the new DataFrame
    summary_str = '~~'.join(summary_concatenated).strip()
    emotion_str = '~~'.join(emotion_concatenated).strip()
    new_df = new_df.append({'topic': group_name, 'summary': summary_str, 'title': emotion_str}, ignore_index=True)

In [37]:
new_df

,topic,summary,title
0,0.0,A customer telling a customer service represen...,I swear I’m not usually this dumb: when the ca...
1,0.0,"Many years ago, the writer worked in a call ce...",A Moment of perfect Existential Dread.~~So you...
2,0.0,The writer had a call with a person who was ve...,Told a customer to suck a dick~~My favorite ca...
3,0.0,A call center agent working for a hotel phone ...,"""Nothing EVER goes to my spam!""~~""Sorry I aske..."
4,0.0,The writer works in a call center for a nation...,Asking for an email address is apparently quit...
...,...,...,...
93,18.0,A 60+ year old man calls an energy company req...,You're a liar!!!~~Tis the season of customers ...
94,18.0,As a tech support worker for residential water...,Contact a scientist because I can't help you.~...
95,19.0,The article describes a customer service repre...,The lawyer card~~Can I speak to someone from t...
96,19.0,A former call center employee shares a frustra...,Call center malicious compliance.~~I’ll wait a...


#### Iterate through the combined summaries, summarize them, generate theme

In [38]:
from tqdm import tqdm

prompt = 'Summarize into a 6 or 7 sentence paragraph.'
count = 1

copy_new_df = new_df.copy()

# Create a new list to store the summaries
new_summaries = []

# Use tqdm to add a progress bar
for index, row in tqdm(new_df.iterrows(), total=len(new_df)):
    rtext = row.get("summary", "")
    rtitle = ""

    try:
        rresponse = getOpenAI_Summary(prompt, rtext)
    except Exception as e:
        print(f"Error generating summary for row {index}: {e}")
        rresponse = ""

    new_summaries.append(rresponse)

    count += 1

# Assign the list of summaries to the DataFrame
copy_new_df["new_summary"] = new_summaries


100%|██████████| 98/98 [09:27<00:00,  5.79s/it]


In [47]:
copy_new_df

,topic,summary,title,new_summary
0,0.0,A customer telling a customer service represen...,I swear I’m not usually this dumb: when the ca...,These paragraphs are a collection of anecdotes...
1,0.0,"Many years ago, the writer worked in a call ce...",A Moment of perfect Existential Dread.~~So you...,This article is a compilation of real-life exp...
2,0.0,The writer had a call with a person who was ve...,Told a customer to suck a dick~~My favorite ca...,This text contains various stories involving c...
3,0.0,A call center agent working for a hotel phone ...,"""Nothing EVER goes to my spam!""~~""Sorry I aske...",Several call center employees share their expe...
4,0.0,The writer works in a call center for a nation...,Asking for an email address is apparently quit...,The text consists of various anecdotes from pe...
...,...,...,...,...
93,18.0,A 60+ year old man calls an energy company req...,You're a liar!!!~~Tis the season of customers ...,These are a series of anecdotes about experien...
94,18.0,As a tech support worker for residential water...,Contact a scientist because I can't help you.~...,The provided text contains various anecdotes a...
95,19.0,The article describes a customer service repre...,The lawyer card~~Can I speak to someone from t...,The article contains various anecdotes of chal...
96,19.0,A former call center employee shares a frustra...,Call center malicious compliance.~~I’ll wait a...,The collection of stories from call center emp...


In [ ]:
from tqdm import tqdm

new_prompt = "This text is a summary of subreddit posts from call center agents.  Describe the post using 3 or 4 words to describe a theme.  Do not elaborate further.  Avoid general terms like customer challenges or customer service or call center.  Some examples are insurance customer challenges, stresses of customer care, racial issues, language barriers:"
count = 1

final_group = copy_new_df.copy()

# Create a new list to store the summaries
new_summaries2 = []

# Use tqdm to add a progress bar
for index, row in tqdm(copy_new_df.iterrows(), total=len(copy_new_df)):
    rtext = row.get("summary", "")
    rtitle = ""

    try:
        rresponse = getOpenAI_Summary(new_prompt, rtext)
    except Exception as e:
        print(f"Error generating summary for row {index}: {e}")
        rresponse = ""

    new_summaries2.append(rresponse)

    print(rresponse)

    count += 1

# Assign the list of summaries to the DataFrame
final_group["theme"] = new_summaries2

In [49]:
df = pd.read_csv('tfcc_submissions_topic_counts_top_20')


In [50]:
import pandas as pd

# Merge the DataFrames on the columns containing topics
merged_df = final_group.merge(df, left_on='topic', right_on='Topic', how='left')

# Create a new column 'topic_name' with the value from the 'Name' column in df
merged_df['topic_name'] = merged_df['Name']

# Drop unnecessary columns
merged_df.drop(['Topic', 'Name'], axis=1, inplace=True)

# Update final_grouped with the new DataFrame
final_group = merged_df

In [51]:
final_group

,topic,summary,title,new_summary,theme,Count,topic_name
0,0.0,A customer telling a customer service represen...,I swear I’m not usually this dumb: when the ca...,These paragraphs are a collection of anecdotes...,Amusing customer interactions.,796,0_like_help_name_say
1,0.0,"Many years ago, the writer worked in a call ce...",A Moment of perfect Existential Dread.~~So you...,This article is a compilation of real-life exp...,Difficult Customers and Interactions,796,0_like_help_name_say
2,0.0,The writer had a call with a person who was ve...,Told a customer to suck a dick~~My favorite ca...,This text contains various stories involving c...,Customer Behavior Issues,796,0_like_help_name_say
3,0.0,A call center agent working for a hotel phone ...,"""Nothing EVER goes to my spam!""~~""Sorry I aske...",Several call center employees share their expe...,Aggressive and Insulting Customers,796,0_like_help_name_say
4,0.0,The writer works in a call center for a nation...,Asking for an email address is apparently quit...,The text consists of various anecdotes from pe...,Difficult customer encounters.,796,0_like_help_name_say
...,...,...,...,...,...,...,...
93,18.0,A 60+ year old man calls an energy company req...,You're a liar!!!~~Tis the season of customers ...,These are a series of anecdotes about experien...,Difficult Customer Interactions.,59,18_meter_heater_complaint_manager
94,18.0,As a tech support worker for residential water...,Contact a scientist because I can't help you.~...,The provided text contains various anecdotes a...,Customer attitude issues,59,18_meter_heater_complaint_manager
95,19.0,The article describes a customer service repre...,The lawyer card~~Can I speak to someone from t...,The article contains various anecdotes of chal...,Customer Anger and Demands,55,19_supervisor_manager_escalated_get
96,19.0,A former call center employee shares a frustra...,Call center malicious compliance.~~I’ll wait a...,The collection of stories from call center emp...,Difficult Customer Escalations,55,19_supervisor_manager_escalated_get


In [52]:
final_group.to_csv('tfcc_top_20_summaries_of_summaries.csv', index=False)

In [3]:
df = pd.read_csv('tfcc_top_20_summaries_of_summaries.csv')


In [4]:
topics = pd.read_csv('tfcc_top20_topics_with_sentiment_and_comments_sentiment.csv')




In [ ]:
# Group the tfcc_comments_with_topics DataFrame by the 'topic' column
grouped = df.groupby('topic')

# Initialize the new DataFrame with empty lists
new_df = pd.DataFrame({'topic': [], 'new_summary': [], 'title': []})

# Iterate through each group in the grouped DataFrame
for group_name, group_df in grouped:
    # Concatenate the 'summary' and 'emotion' values in the group with '~~' separator
    summary_list = group_df['new_summary'].tolist()
    emotion_list = group_df['title'].tolist()
    summary_concatenated = []
    emotion_concatenated = []
    for summary, emotion in zip(summary_list, emotion_list):
        # Check if the concatenated summary would exceed 2000 words
        words_summary = ' '.join(summary_concatenated).split()
        if len(words_summary) + len(summary.split()) > 2000:
            # Add the current concatenated summary and emotion to the new DataFrame
            summary_str = '~~'.join(summary_concatenated).strip()
            emotion_str = '~~'.join(emotion_concatenated).strip()
            new_df = new_df.append({'topic': group_name, 'new_summary': summary_str, 'title': emotion_str}, ignore_index=True)
            summary_concatenated = []
            emotion_concatenated = []
        # Concatenate the current summary and emotion to the concatenated summary and emotion with '~~' separator
        summary_concatenated.append(summary)
        emotion_str = '~~'.join(emotion) if isinstance(emotion, list) else emotion
        emotion_concatenated.append(emotion_str)
    # Add any remaining concatenated summary and emotion to the new DataFrame
    summary_str = '~~'.join(summary_concatenated).strip()
    emotion_str = '~~'.join(emotion_concatenated).strip()
    new_df = new_df.append({'topic': group_name, 'new_summary': summary_str, 'title': emotion_str}, ignore_index=True)

In [9]:
new_df

,topic,new_summary,title
0,0.0,A collection of anecdotes and experiences from...,I swear I’m not usually this dumb: when the ca...
1,1.0,These subreddit posts share anecdotes from cus...,Is. Your. Card. Damaged?!~~A scam that shook m...
2,2.0,"A collection of customer service stories, incl...",We can hear you when you’re on hold. Say goodb...
3,3.0,These subreddit posts feature a variety of int...,Don't cheat on your wife~~Insurance fraud ma’a...
4,4.0,Call center employees share various frustratin...,You're telling me I have to wear a mask to go ...
5,5.0,A collection of experiences from employees in ...,"Today on ""I can't believe I had to explain thi..."
6,6.0,A collection of stories from telecom employees...,I will never forget the sound of this customer...
7,7.0,These posts highlight the toll that call cente...,"At 32, I’m getting bullied at work~~Coworker s..."
8,8.0,This text describes several instances of racis...,“I want to speak to somebody who speaks real e...
9,9.0,Many individuals share their experiences of wo...,In which new management fires everyone~~I FINA...


In [12]:
topics

,Topic,Count,Name,avg_pos_sentiment,avg_neg_sentiment,avg_comments_pos_sentiment,avg_comments_neg_sentiment
0,0,796,0_like_help_name_say,0.161216,0.838784,0.196144,0.803856
1,1,591,1_card_bank_credit_fraud,0.113983,0.886017,0.148113,0.851887
2,2,457,2_delivery_shipping_store_email,0.122277,0.877723,0.163302,0.836698
3,3,283,3_insurance_car_claims_coverage,0.103466,0.896534,0.117970,0.882030
4,4,243,4_patient_doctor_clinic_medical,0.139520,0.860480,0.159091,0.840909
5,5,201,5_tow_roadside_truck_assistance,0.159511,0.840489,0.139324,0.860676
6,6,188,6_bill_phones_service_data,0.133500,0.866500,0.103897,0.896103
7,7,108,7_job_feel_anxiety_work,0.154886,0.845114,0.290429,0.709571
8,8,106,8_english_spanish_speak_language,0.154176,0.845824,0.208100,0.791900
9,9,101,9_job_interview_role_experience,0.374143,0.625857,0.343623,0.656377


In [13]:
# Merge the two dataframes using pd.concat()
merged_df = pd.concat([new_df, topics], axis=1)


In [14]:
merged_df

,topic,new_summary,title,Topic,Count,Name,avg_pos_sentiment,avg_neg_sentiment,avg_comments_pos_sentiment,avg_comments_neg_sentiment
0,0.0,A collection of anecdotes and experiences from...,I swear I’m not usually this dumb: when the ca...,0,796,0_like_help_name_say,0.161216,0.838784,0.196144,0.803856
1,1.0,These subreddit posts share anecdotes from cus...,Is. Your. Card. Damaged?!~~A scam that shook m...,1,591,1_card_bank_credit_fraud,0.113983,0.886017,0.148113,0.851887
2,2.0,"A collection of customer service stories, incl...",We can hear you when you’re on hold. Say goodb...,2,457,2_delivery_shipping_store_email,0.122277,0.877723,0.163302,0.836698
3,3.0,These subreddit posts feature a variety of int...,Don't cheat on your wife~~Insurance fraud ma’a...,3,283,3_insurance_car_claims_coverage,0.103466,0.896534,0.117970,0.882030
4,4.0,Call center employees share various frustratin...,You're telling me I have to wear a mask to go ...,4,243,4_patient_doctor_clinic_medical,0.139520,0.860480,0.159091,0.840909
5,5.0,A collection of experiences from employees in ...,"Today on ""I can't believe I had to explain thi...",5,201,5_tow_roadside_truck_assistance,0.159511,0.840489,0.139324,0.860676
6,6.0,A collection of stories from telecom employees...,I will never forget the sound of this customer...,6,188,6_bill_phones_service_data,0.133500,0.866500,0.103897,0.896103
7,7.0,These posts highlight the toll that call cente...,"At 32, I’m getting bullied at work~~Coworker s...",7,108,7_job_feel_anxiety_work,0.154886,0.845114,0.290429,0.709571
8,8.0,This text describes several instances of racis...,“I want to speak to somebody who speaks real e...,8,106,8_english_spanish_speak_language,0.154176,0.845824,0.208100,0.791900
9,9.0,Many individuals share their experiences of wo...,In which new management fires everyone~~I FINA...,9,101,9_job_interview_role_experience,0.374143,0.625857,0.343623,0.656377


In [ ]:
prompt = 'summarize the following text to create a brief introductory paragraph that describes the content and meaning at a high level using academic language:'
count = 1

new_merged_df = merged_df.copy()

# Create a new list to store the summaries
new_summaries = []

for index, row in new_merged_df.iterrows():
    rtext = row["new_summary"]
    rtitle = ""

    rresponse = getOpenAI_Summary(prompt, rtext)

    print(count)
    count = count + 1

    # Append the summary to the list
    new_summaries.append(rresponse)

# Assign the list of summaries to the DataFrame
new_merged_df["intro_summary"] = new_summaries

In [19]:
new_merged_df

,topic,new_summary,title,Topic,Count,Name,avg_pos_sentiment,avg_neg_sentiment,avg_comments_pos_sentiment,avg_comments_neg_sentiment,intro_summary
0,0.0,A collection of anecdotes and experiences from...,I swear I’m not usually this dumb: when the ca...,0,796,0_like_help_name_say,0.161216,0.838784,0.196144,0.803856,The text is a collection of Reddit posts that ...
1,1.0,These subreddit posts share anecdotes from cus...,Is. Your. Card. Damaged?!~~A scam that shook m...,1,591,1_card_bank_credit_fraud,0.113983,0.886017,0.148113,0.851887,The following texts contain anecdotes from cus...
2,2.0,"A collection of customer service stories, incl...",We can hear you when you’re on hold. Say goodb...,2,457,2_delivery_shipping_store_email,0.122277,0.877723,0.163302,0.836698,The articles discuss various experiences of cu...
3,3.0,These subreddit posts feature a variety of int...,Don't cheat on your wife~~Insurance fraud ma’a...,3,283,3_insurance_car_claims_coverage,0.103466,0.896534,0.117970,0.882030,The following text features a collection of an...
4,4.0,Call center employees share various frustratin...,You're telling me I have to wear a mask to go ...,4,243,4_patient_doctor_clinic_medical,0.139520,0.860480,0.159091,0.840909,This text describes a collection of anecdotes ...
5,5.0,A collection of experiences from employees in ...,"Today on ""I can't believe I had to explain thi...",5,201,5_tow_roadside_truck_assistance,0.159511,0.840489,0.139324,0.860676,The following texts contain collections of sto...
6,6.0,A collection of stories from telecom employees...,I will never forget the sound of this customer...,6,188,6_bill_phones_service_data,0.133500,0.866500,0.103897,0.896103,This text discusses various experiences of emp...
7,7.0,These posts highlight the toll that call cente...,"At 32, I’m getting bullied at work~~Coworker s...",7,108,7_job_feel_anxiety_work,0.154886,0.845114,0.290429,0.709571,This article discusses the challenges faced by...
8,8.0,This text describes several instances of racis...,“I want to speak to somebody who speaks real e...,8,106,8_english_spanish_speak_language,0.154176,0.845824,0.208100,0.791900,"The text discusses instances of racism, xenoph..."
9,9.0,Many individuals share their experiences of wo...,In which new management fires everyone~~I FINA...,9,101,9_job_interview_role_experience,0.374143,0.625857,0.343623,0.656377,The text discusses individuals' experiences of...


In [20]:
new_merged_df.to_csv('tfcc_summary_details.csv', index=False)